In [1]:
import pandas as pd

In [2]:
df = pd.read_csv(r'C:/Work/DataFrames/100000_UK_used_car/bmw.csv')

In [3]:
df.head()

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
0,5 Series,2014,11200,Automatic,67068,Diesel,125,57.6,2.0
1,6 Series,2018,27000,Automatic,14827,Petrol,145,42.8,2.0
2,5 Series,2016,16000,Automatic,62794,Diesel,160,51.4,3.0
3,1 Series,2017,12750,Automatic,26676,Diesel,145,72.4,1.5
4,7 Series,2014,14500,Automatic,39554,Diesel,160,50.4,3.0


In [4]:
from sklearn.model_selection import train_test_split

### Тестовая и тренировачная выборки

In [5]:
train, test = train_test_split(df,train_size=0.6,random_state=42)

In [6]:
len(train) / len(df)

0.5999443465355718

In [7]:
len(test) / len(df)

0.4000556534644282

### Валидационная выборка

In [8]:
val, test = train_test_split(test,train_size=0.5,random_state=42)

In [9]:
len(test) / len(df)

0.20007420461923756

In [10]:
len(val) / len(df)

0.19998144884519062

In [11]:
len(train) / len(df)

0.5999443465355718

### Список фичей

In [12]:
train.columns

Index(['model', 'year', 'price', 'transmission', 'mileage', 'fuelType', 'tax',
       'mpg', 'engineSize'],
      dtype='object')

In [13]:
X = ['model', 'year', 'transmission', 'mileage', 'fuelType', 'tax', 'mpg', 'engineSize']

y = ['price']

cat_features = ['model', 'transmission', 'fuelType']    #указываем, что здесь текстовые фичи

### Принцип преобразования категориальных фичей

In [14]:
train['transmission']

5133    Semi-Auto
2121    Automatic
6576    Semi-Auto
8330       Manual
3161    Semi-Auto
          ...    
5734    Semi-Auto
5191    Semi-Auto
5390    Semi-Auto
860     Semi-Auto
7270    Automatic
Name: transmission, Length: 6468, dtype: object

In [15]:
pd.get_dummies(train['transmission'])

,Automatic,Manual,Semi-Auto
5133,0,0,1
2121,1,0,0
6576,0,0,1
8330,0,1,0
3161,0,0,1
...,...,...,...
5734,0,0,1
5191,0,0,1
5390,0,0,1
860,0,0,1


https://catboost.ai/en/docs/concepts/algorithm-main-stages_cat-to-numberic

## Первый запуск Кэтбуста

In [16]:
from catboost import CatBoostRegressor

In [18]:
X = ['year', 'transmission', 'engineSize']

y = ['price']

cat_features = [ 'transmission']    #указываем, что здесь текстовые фичи

In [22]:
model = CatBoostRegressor(cat_features=cat_features,
                          eval_metric='MAPE',
                          random_seed=42,
                          verbose=100)

In [23]:
model.fit(train[X],train[y], eval_set=(val[X],val[y]))

Learning rate set to 0.068263
0:	learn: 0.4603375	test: 0.4555509	best: 0.4555509 (0)	total: 39.7ms	remaining: 39.7s
100:	learn: 0.1546393	test: 0.1529687	best: 0.1529687 (100)	total: 3.66s	remaining: 32.6s
200:	learn: 0.1528213	test: 0.1517332	best: 0.1517332 (200)	total: 6.91s	remaining: 27.5s
300:	learn: 0.1517532	test: 0.1514386	best: 0.1514000 (288)	total: 10.6s	remaining: 24.7s
400:	learn: 0.1509722	test: 0.1515508	best: 0.1514000 (288)	total: 14.4s	remaining: 21.5s
500:	learn: 0.1502967	test: 0.1516166	best: 0.1514000 (288)	total: 18.1s	remaining: 18.1s
600:	learn: 0.1499412	test: 0.1516849	best: 0.1514000 (288)	total: 22.1s	remaining: 14.6s
700:	learn: 0.1494768	test: 0.1516928	best: 0.1514000 (288)	total: 25.2s	remaining: 10.8s
800:	learn: 0.1490851	test: 0.1518297	best: 0.1514000 (288)	total: 28s	remaining: 6.95s
900:	learn: 0.1487173	test: 0.1519119	best: 0.1514000 (288)	total: 30.7s	remaining: 3.37s
999:	learn: 0.1483869	test: 0.1519628	best: 0.1514000 (288)	total: 33.4s	re

In [25]:
test[X]

,year,transmission,engineSize
4984,2019,Semi-Auto,3.0
4880,2018,Semi-Auto,2.0
9132,2016,Automatic,2.0
6451,2020,Semi-Auto,2.0
7199,2020,Semi-Auto,2.0
...,...,...,...
9627,2015,Automatic,3.0
6096,2019,Automatic,3.0
1747,2019,Automatic,2.0
4023,2016,Automatic,2.0


In [24]:
model.predict(test[X])

array([45436.28801924, 21738.83525716, 16205.32627896, ...,
       28444.11570854, 16205.32627896, 27824.78614511])

In [26]:
test['price_pred'] = model.predict(test[X])

In [27]:
test.head()

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize,price_pred
4984,X6,2019,52990,Semi-Auto,3086,Diesel,145,34.9,3.0,45436.288019
4880,X1,2018,24081,Semi-Auto,13245,Diesel,150,60.1,2.0,21738.835257
9132,1 Series,2016,12999,Automatic,68949,Petrol,200,43.5,2.0,16205.326279
6451,1 Series,2020,11995,Semi-Auto,10,Petrol,150,34.5,2.0,32088.707415
7199,3 Series,2020,29875,Semi-Auto,150,Petrol,145,42.2,2.0,32088.707415


In [28]:
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

In [29]:
def error(y_true,y_pred):
    print(mean_absolute_error(y_true,y_pred))
    print(mean_absolute_percentage_error(y_true,y_pred))

In [30]:
error(test['price'],test['price_pred'])

3366.679482256595
0.15363582890850797


## Обучаемся на всех фичах

In [31]:
X = ['model', 'year', 'transmission', 'mileage', 'fuelType', 'tax',
       'mpg', 'engineSize']

y = ['price']

cat_features = ['model', 'transmission', 'fuelType']

In [32]:
parameters = {'cat_features': cat_features,
              'eval_metric': 'MAPE',
              'random_seed':42,
              'verbose':100}

In [33]:
model = CatBoostRegressor(**parameters)

In [34]:
model.fit(train[X],train[y],eval_set=(val[X],val[y]))

Learning rate set to 0.068263
0:	learn: 0.4607438	test: 0.4563750	best: 0.4563750 (0)	total: 36.2ms	remaining: 36.1s
100:	learn: 0.0904302	test: 0.0896388	best: 0.0896388 (100)	total: 4.39s	remaining: 39s
200:	learn: 0.0781784	test: 0.0803125	best: 0.0803125 (200)	total: 8.99s	remaining: 35.7s
300:	learn: 0.0729925	test: 0.0764506	best: 0.0764506 (300)	total: 13.9s	remaining: 32.2s
400:	learn: 0.0695552	test: 0.0744676	best: 0.0744553 (399)	total: 17.3s	remaining: 25.9s
500:	learn: 0.0670683	test: 0.0732827	best: 0.0732827 (500)	total: 20.9s	remaining: 20.8s
600:	learn: 0.0652835	test: 0.0726170	best: 0.0726128 (597)	total: 24.5s	remaining: 16.3s
700:	learn: 0.0636320	test: 0.0720622	best: 0.0720500 (695)	total: 27.9s	remaining: 11.9s
800:	learn: 0.0623408	test: 0.0714586	best: 0.0714329 (794)	total: 31.2s	remaining: 7.76s
900:	learn: 0.0611487	test: 0.0711564	best: 0.0711321 (898)	total: 34.4s	remaining: 3.78s
999:	learn: 0.0601407	test: 0.0708439	best: 0.0708403 (998)	total: 37.8s	re

In [35]:
test['price_pred_all'] = model.predict(test[X])

In [36]:
error(test['price'],test['price_pred_all'])

1556.5378327680894
0.07367840910148211


# Количество итераций и learning rate

### Уменьшим learning rate

In [37]:
X = ['year', 'transmission', 'engineSize']

cat_features = ['transmission']

y = ['price']

In [38]:
model = CatBoostRegressor(cat_features=cat_features,
                          learning_rate = 0.03,
                          eval_metric='MAPE',
                          random_seed=42,
                          verbose=100)

In [39]:
model.fit(train[X],train[y],eval_set=(val[X],val[y]))

0:	learn: 0.4746987	test: 0.4699730	best: 0.4699730 (0)	total: 36.1ms	remaining: 36s
100:	learn: 0.1681230	test: 0.1638970	best: 0.1638970 (100)	total: 3.9s	remaining: 34.7s
200:	learn: 0.1561398	test: 0.1538311	best: 0.1538311 (200)	total: 7.49s	remaining: 29.8s
300:	learn: 0.1543096	test: 0.1525317	best: 0.1525309 (299)	total: 10.5s	remaining: 24.3s
400:	learn: 0.1534465	test: 0.1519221	best: 0.1519198 (397)	total: 13s	remaining: 19.3s
500:	learn: 0.1527824	test: 0.1516330	best: 0.1516164 (480)	total: 15.8s	remaining: 15.7s
600:	learn: 0.1522590	test: 0.1514284	best: 0.1514284 (600)	total: 18.5s	remaining: 12.3s
700:	learn: 0.1518052	test: 0.1513704	best: 0.1513673 (689)	total: 21.2s	remaining: 9.04s
800:	learn: 0.1514495	test: 0.1513967	best: 0.1513651 (703)	total: 23.9s	remaining: 5.95s
900:	learn: 0.1510958	test: 0.1513657	best: 0.1513489 (873)	total: 26.7s	remaining: 2.93s
999:	learn: 0.1508297	test: 0.1513297	best: 0.1513168 (992)	total: 29.4s	remaining: 0us

bestTest = 0.151316

### Увеличим learning rate

In [40]:
X = ['model', 'year', 'transmission', 'mileage', 'fuelType', 'tax',
       'mpg', 'engineSize']

cat_features = ['model', 'transmission', 'fuelType']

y = ['price']

In [41]:
parameters = {'cat_features': cat_features,
              'eval_metric': 'MAPE',
              'learning_rate': 0.08,
              'random_seed':42,
              'verbose':100}

In [42]:
model = CatBoostRegressor(**parameters)

In [43]:
model.fit(train[X],train[y],eval_set=(val[X],val[y]))

0:	learn: 0.4564416	test: 0.4521294	best: 0.4521294 (0)	total: 35.3ms	remaining: 35.3s
100:	learn: 0.0875290	test: 0.0867329	best: 0.0867329 (100)	total: 4.68s	remaining: 41.7s
200:	learn: 0.0760834	test: 0.0779235	best: 0.0779235 (200)	total: 8.64s	remaining: 34.3s
300:	learn: 0.0708670	test: 0.0740290	best: 0.0740290 (300)	total: 12.4s	remaining: 28.7s
400:	learn: 0.0676240	test: 0.0718814	best: 0.0718814 (400)	total: 15.8s	remaining: 23.6s
500:	learn: 0.0654070	test: 0.0709221	best: 0.0708594 (496)	total: 19.4s	remaining: 19.3s
600:	learn: 0.0638577	test: 0.0704606	best: 0.0704490 (599)	total: 22.8s	remaining: 15.1s
700:	learn: 0.0622969	test: 0.0698226	best: 0.0698226 (700)	total: 26.3s	remaining: 11.2s
800:	learn: 0.0609583	test: 0.0694169	best: 0.0694004 (791)	total: 29.7s	remaining: 7.37s
900:	learn: 0.0596089	test: 0.0689968	best: 0.0689945 (895)	total: 33.3s	remaining: 3.66s
999:	learn: 0.0584626	test: 0.0686829	best: 0.0686377 (993)	total: 36.6s	remaining: 0us

bestTest = 0.0

## Останавливаем, если нет улучшения


In [44]:
X = ['year', 'transmission', 'engineSize']

cat_features = ['transmission']

y = ['price']


In [45]:
model = CatBoostRegressor(cat_features=cat_features,
                          early_stopping_rounds=200,
                          eval_metric='MAPE',
                          random_seed=42,
                          verbose=100)

In [46]:
model.fit(train[X],train[y],eval_set=(val[X],val[y]))

Learning rate set to 0.068263
0:	learn: 0.4603375	test: 0.4555509	best: 0.4555509 (0)	total: 35.6ms	remaining: 35.6s
100:	learn: 0.1546393	test: 0.1529687	best: 0.1529687 (100)	total: 3.83s	remaining: 34.1s
200:	learn: 0.1528213	test: 0.1517332	best: 0.1517332 (200)	total: 7.34s	remaining: 29.2s
300:	learn: 0.1517532	test: 0.1514386	best: 0.1514000 (288)	total: 11.6s	remaining: 26.9s
400:	learn: 0.1509722	test: 0.1515508	best: 0.1514000 (288)	total: 14.9s	remaining: 22.2s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.1514000297
bestIteration = 288

Shrink model to first 289 iterations.


# Обучаемся на всех данных

In [47]:
len(train)

6468

In [48]:
len(val)

2156

In [49]:
len(train) + len(val)

8624

In [50]:
train_full = pd.concat([train,val])

In [51]:
train_full.head()

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
5133,X6,2015,25000,Semi-Auto,26100,Diesel,205,45.6,3.0
2121,X2,2019,31980,Automatic,4804,Petrol,145,34.0,2.0
6576,X2,2018,22995,Semi-Auto,24136,Petrol,145,38.7,2.0
8330,3 Series,2016,12999,Manual,52224,Diesel,125,62.8,2.0
3161,X3,2019,32980,Semi-Auto,1961,Diesel,150,54.3,2.0


In [52]:
X = ['model', 'year', 'transmission', 'mileage', 'fuelType', 'tax',
       'mpg', 'engineSize']

cat_features = ['model', 'transmission', 'fuelType']

y = ['price']

In [53]:
parameters = {'cat_features': cat_features,
              'eval_metric': 'MAPE',
              'learning_rate': 0.08,
              'random_seed':42,
              'verbose':100}

In [54]:
model = CatBoostRegressor(**parameters)

In [55]:
model.fit(train[X],train[y],eval_set=(val[X],val[y]))

0:	learn: 0.4564416	test: 0.4521294	best: 0.4521294 (0)	total: 36.3ms	remaining: 36.3s
100:	learn: 0.0875290	test: 0.0867329	best: 0.0867329 (100)	total: 4.19s	remaining: 37.3s
200:	learn: 0.0760834	test: 0.0779235	best: 0.0779235 (200)	total: 8.87s	remaining: 35.2s
300:	learn: 0.0708670	test: 0.0740290	best: 0.0740290 (300)	total: 12.7s	remaining: 29.5s
400:	learn: 0.0676240	test: 0.0718814	best: 0.0718814 (400)	total: 17.4s	remaining: 26s
500:	learn: 0.0654070	test: 0.0709221	best: 0.0708594 (496)	total: 20.9s	remaining: 20.8s
600:	learn: 0.0638577	test: 0.0704606	best: 0.0704490 (599)	total: 24.4s	remaining: 16.2s
700:	learn: 0.0622969	test: 0.0698226	best: 0.0698226 (700)	total: 28s	remaining: 12s
800:	learn: 0.0609583	test: 0.0694169	best: 0.0694004 (791)	total: 31.5s	remaining: 7.82s
900:	learn: 0.0596089	test: 0.0689968	best: 0.0689945 (895)	total: 35s	remaining: 3.85s
999:	learn: 0.0584626	test: 0.0686829	best: 0.0686377 (993)	total: 38.7s	remaining: 0us

bestTest = 0.068637745

In [56]:
model.best_iteration_

993

In [57]:
parameters = {'iterations': model.best_iteration_ + 1,
              'cat_features': cat_features,
              'eval_metric': 'MAPE',
              'learning_rate': 0.08,
              'random_seed':42,
              'verbose':100}

In [58]:
parameters

{'iterations': 994,
 'cat_features': ['model', 'transmission', 'fuelType'],
 'eval_metric': 'MAPE',
 'learning_rate': 0.08,
 'random_seed': 42,
 'verbose': 100}

In [59]:
model = CatBoostRegressor(**parameters)

In [60]:
model.fit(train_full[X],train_full[y])

0:	learn: 0.4538016	total: 27.8ms	remaining: 27.6s
100:	learn: 0.0854178	total: 3.3s	remaining: 29.2s
200:	learn: 0.0745022	total: 6.79s	remaining: 26.8s
300:	learn: 0.0701717	total: 10.4s	remaining: 23.9s
400:	learn: 0.0676577	total: 15.2s	remaining: 22.5s
500:	learn: 0.0656926	total: 20s	remaining: 19.7s
600:	learn: 0.0638104	total: 24.3s	remaining: 15.9s
700:	learn: 0.0624302	total: 27.6s	remaining: 11.6s
800:	learn: 0.0612275	total: 30.9s	remaining: 7.45s
900:	learn: 0.0599823	total: 34.3s	remaining: 3.54s
993:	learn: 0.0590330	total: 37.4s	remaining: 0us


In [61]:
test['price_pred_all_features_and_data'] = model.predict(test[X])

In [62]:
error(test['price'],test['price_pred_all'])

1556.5378327680894
0.07367840910148211


In [63]:
error(test['price'],test['price_pred_all_features_and_data'])

1512.6160783572977
0.07305203023986592


## Метрики регрессии

https://catboost.ai/en/docs/concepts/loss-functions-regression#objectives-and-metrics

## Меняем функцию оптимизации

In [64]:
X = ['model', 'year', 'transmission', 'mileage', 'fuelType', 'tax',
       'mpg', 'engineSize']

cat_features = ['model', 'transmission', 'fuelType']

y = ['price']

In [65]:
parameters = {'cat_features': cat_features,
              'loss_function': 'MAE',
              'eval_metric': 'MAPE',
              'learning_rate': 0.1,
              'random_seed':42,
              'verbose':100}

In [66]:
model = CatBoostRegressor(**parameters)

In [67]:
model.fit(train[X],train[y],eval_set=(val[X],val[y]))

0:	learn: 0.3849345	test: 0.3793341	best: 0.3793341 (0)	total: 41.5ms	remaining: 41.5s
100:	learn: 0.0764009	test: 0.0774578	best: 0.0774578 (100)	total: 4.82s	remaining: 42.9s
200:	learn: 0.0666216	test: 0.0706498	best: 0.0706498 (200)	total: 9.5s	remaining: 37.8s
300:	learn: 0.0628577	test: 0.0695086	best: 0.0694924 (298)	total: 14.5s	remaining: 33.6s
400:	learn: 0.0605748	test: 0.0687890	best: 0.0687826 (399)	total: 19.5s	remaining: 29.1s
500:	learn: 0.0589764	test: 0.0683467	best: 0.0683222 (490)	total: 24.5s	remaining: 24.4s
600:	learn: 0.0579201	test: 0.0680567	best: 0.0680567 (600)	total: 29.4s	remaining: 19.5s
700:	learn: 0.0570454	test: 0.0678671	best: 0.0678659 (681)	total: 34.1s	remaining: 14.5s
800:	learn: 0.0563072	test: 0.0676018	best: 0.0676018 (800)	total: 38.9s	remaining: 9.66s
900:	learn: 0.0555798	test: 0.0674500	best: 0.0674384 (889)	total: 43.6s	remaining: 4.79s
999:	learn: 0.0548863	test: 0.0673326	best: 0.0672972 (972)	total: 47.8s	remaining: 0us

bestTest = 0.06